In [28]:
import os
import glob
import shutil
import numpy as np

import xml.etree.ElementTree as ET
import re

In [142]:
data_path = "datasets/road"
annotations_path = "datasets/road/Annotations"

labels = ["person", "car", "bus"]

def element_decode(elem_name):
    elem_decode = ET.tostring(elem_name).decode()
    obj_name = re.search('</(.*)>', elem_decode).group(1)
    
    return obj_name

def darknet_coord(bounding_box, image_size):
    print(bounding_box)
#     x_width = int(bounding_box[2]) - int(bounding_box[0])
#     y_width = int(bounding_box[3]) - int(bounding_box[1])
        
#     x_width_abs = x_width / int(image_size[1])
#     y_width_abs = y_width / int(image_size[0])
    
#     x_centre = (x_width + int(bounding_box[0])) / 2
#     y_centre = (y_width + int(bounding_box[1])) / 2

#     x_centre_abs = (x_centre) / int(image_size[1])
#     y_centre_abs = (y_centre) / int(image_size[0])
    
#     return x_centre_abs, y_centre_abs, x_width_abs, y_width_abs
    
    # scaling coords first, then doing maths? 
    
    xmin = float(bounding_box[0])
    ymin = float(bounding_box[1])
    xmax = float(bounding_box[2])
    ymax = float(bounding_box[3])
    
    image_height = float(image_size[0])
    image_width = float(image_size[1])

    boundbox_height = ymax - ymin
    boundbox_width = xmax - xmin
    
    width = boundbox_width / image_width
    height = boundbox_height / image_height
    
    x_centre = (xmax+xmin) / 2 / image_width
    y_centre = (ymax+ymin) / 2 / image_height
        
    return x_centre, y_centre, width, height
    
    
    
    
# look through annotations folder and match 
for file in glob.glob(annotations_path + "/*.xml"):
    print(file)
    
    tree = ET.parse(file)
    root = tree.getroot()
    
    required_item = 0
    curr_item = ""
    curr_obj = []
    for elem in root:
        for subelem in elem:
            obj_name = subelem.text
            if obj_name in labels:
                curr_item = obj_name
            decode_elem_name = ET.tostring(subelem).decode()
            
            if "bndbox" in decode_elem_name and curr_item in labels:
                    #print(curr_item, decode_elem_name)
                    # bounding coordinates
                    xmin = re.findall(r'<xmin>(.+?)</xmin>', decode_elem_name)[0]
                    xmax = re.findall(r'<xmax>(.+?)</xmax>', decode_elem_name)[0]
                
                    ymin = re.findall(r'<ymin>(.+?)</ymin>', decode_elem_name)[0]
                    ymax = re.findall(r'<ymax>(.+?)</ymax>', decode_elem_name)[0]

                    #print(xmin, ymin, xmax, ymax)
                    curr_obj.append(curr_item)
                    curr_obj.append([xmin, ymin, xmax, ymax])
                    #print(curr_obj)
            
        tag_name = ET.tostring(elem).decode()
        if "size" in tag_name:
            if len(curr_obj) > 0:
                if curr_obj[0] in labels:
                    height = re.findall(r'<height>(.+?)</height>', tag_name)[0]
                    width = re.findall(r'<width>(.+?)</width>', tag_name)[0]

                    curr_obj.append([height, width])
                    #print(curr_obj)
                    
                    # create new text file containing the 
                    image_name = file.split("/")[3].split(".")[0]
                    annotation_text_path = "datasets/road/TextAnnotations/" + image_name + ".txt"
                                        
                    file_object  = open(annotation_text_path, "w+") 
                    
                    # values to be filled in 
                    obj_label_number = 0
                    image_size = curr_obj[-1]
                    
                    for i in range(len(curr_obj)):
                        curr_obj_item = curr_obj[i]
                        #print(curr_obj_item, len(curr_obj_item))
                
                        if isinstance(curr_obj_item, str) == True:
                            obj_label_number = labels.index(curr_obj_item)
                        
                        if isinstance(curr_obj_item, list) == True and len(curr_obj_item) == 4:
                            #print(obj_label_number, curr_obj_item, image_size)
                            dn_coords = darknet_coord(curr_obj_item, image_size)
                            #print(obj_label_number, dn_coords)
                        
                            obj_item = (str(obj_label_number) + " " + str(dn_coords[0])
                                       + " " + str(dn_coords[1]) + " " + str(dn_coords[2])
                                       + " " + str(dn_coords[3]) + "\n")
                            print(obj_item)
                            
                            file_object.write(obj_item)
                            
                    print(annotation_text_path)

#             for subsubelem in elem:
#                 decode_elem_name = element_decode(subsubelem)
#                 print(decode_elem_name)
#                 if decode_elem_name == "bndbox":
#                     print(subsubelem.text)
            #print(decode_obj_name, subelem.text)
            #print(subelem.text)

#     with open(file,'r') as f:
#         output = f.read()
#         # if file is empty, remove the text file 
#         if len(output) == 0:
#             shutil.move(file, "datasets/paris_old")

datasets/road/Annotations/2010_004276.xml
datasets/road/Annotations/2010_000014.xml
datasets/road/Annotations/2009_001780.xml
datasets/road/Annotations/2009_004397.xml
datasets/road/Annotations/2008_004872.xml
datasets/road/Annotations/2007_008802.xml
datasets/road/Annotations/2010_004691.xml
datasets/road/Annotations/2010_006329.xml
['290', '175', '337', '299']
0 0.627 0.7117117117117117 0.094 0.37237237237237236

datasets/road/TextAnnotations/2010_006329.txt
datasets/road/Annotations/2009_003902.xml
datasets/road/Annotations/2008_006621.xml
datasets/road/Annotations/2012_002745.xml
['94', '23', '439', '336']
0 0.533 0.4786666666666667 0.69 0.8346666666666667

datasets/road/TextAnnotations/2012_002745.txt
datasets/road/Annotations/2009_003848.xml
datasets/road/Annotations/2008_000016.xml
datasets/road/Annotations/2010_004669.xml
datasets/road/Annotations/2012_003441.xml
['131', '148', '329', '500']
0 0.6990881458966566 0.648 0.601823708206687 0.704

datasets/road/TextAnnotations/2012_

datasets/road/Annotations/2012_003663.xml
['217', '282', '263', '360']
0 0.48 0.856 0.092 0.208

datasets/road/TextAnnotations/2012_003663.txt
datasets/road/Annotations/2011_000505.xml
['320', '215', '368', '241']
1 0.688 0.608 0.096 0.06933333333333333

['370', '215', '406', '240']
1 0.776 0.6066666666666667 0.072 0.06666666666666667

['252', '214', '293', '244']
1 0.545 0.6106666666666667 0.082 0.08

['1', '210', '71', '247']
1 0.072 0.6093333333333333 0.14 0.09866666666666667

['16', '214', '97', '248']
1 0.113 0.616 0.162 0.09066666666666667

['96', '112', '194', '325']
0 0.29 0.5826666666666667 0.196 0.568

['267', '217', '280', '250']
0 0.547 0.6226666666666667 0.026 0.088

['464', '209', '476', '250']
0 0.94 0.612 0.024 0.10933333333333334

['456', '213', '469', '250']
0 0.925 0.6173333333333333 0.026 0.09866666666666667

['447', '210', '458', '237']
0 0.905 0.596 0.022 0.072

['302', '216', '325', '232']
1 0.627 0.5973333333333334 0.046 0.042666666666666665

datasets/road/TextA

['48', '43', '323', '449']
0 0.5570570570570571 0.492 0.8258258258258259 0.812

datasets/road/TextAnnotations/2011_003488.txt
datasets/road/Annotations/2010_003875.xml
datasets/road/Annotations/2011_005822.xml
['48', '66', '244', '362']
0 0.43843843843843844 0.428 0.5885885885885885 0.592

datasets/road/TextAnnotations/2011_005822.txt
datasets/road/Annotations/2009_002431.xml
datasets/road/Annotations/2007_009655.xml
datasets/road/Annotations/2010_005712.xml
datasets/road/Annotations/2008_006626.xml
datasets/road/Annotations/2012_001966.xml
['39', '80', '224', '375']
0 0.263 0.6066666666666667 0.37 0.7866666666666666

['133', '90', '300', '375']
0 0.433 0.62 0.334 0.76

['276', '85', '424', '375']
0 0.7 0.6133333333333333 0.296 0.7733333333333333

datasets/road/TextAnnotations/2012_001966.txt
datasets/road/Annotations/2010_004289.xml
['199', '163', '244', '221']
0 0.6651651651651652 0.384 0.13513513513513514 0.116

['37', '231', '213', '387']
0 0.37537537537537535 0.618 0.5285285285285

['296', '208', '500', '375']
0 0.796 0.7773333333333333 0.408 0.44533333333333336

['60', '233', '233', '375']
0 0.293 0.8106666666666666 0.346 0.37866666666666665

datasets/road/TextAnnotations/2010_005293.txt
datasets/road/Annotations/2011_004534.xml
['148', '115', '271', '375']
0 0.419 0.6533333333333333 0.246 0.6933333333333334

datasets/road/TextAnnotations/2011_004534.txt
datasets/road/Annotations/2010_000293.xml
datasets/road/Annotations/2012_004210.xml
['62', '32', '217', '300']
0 0.5513833992094862 0.5533333333333333 0.6126482213438735 0.8933333333333333

datasets/road/TextAnnotations/2012_004210.txt
datasets/road/Annotations/2012_003113.xml
['99', '111', '221', '347']
0 0.32 0.6106666666666667 0.244 0.6293333333333333

datasets/road/TextAnnotations/2012_003113.txt
datasets/road/Annotations/2007_004707.xml
datasets/road/Annotations/2009_001103.xml
['137', '71', '354', '331']
0 0.491 0.536 0.434 0.6933333333333334

['272', '276', '310', '375']
0 0.582 0.868 0.076 0.264

['371',

['269', '57', '379', '375']
0 0.648 0.576 0.22 0.848

['132', '81', '263', '375']
0 0.395 0.608 0.262 0.784

['386', '253', '426', '285']
0 0.812 0.7173333333333334 0.08 0.08533333333333333

datasets/road/TextAnnotations/2009_001934.txt
datasets/road/Annotations/2008_003774.xml
datasets/road/Annotations/2008_008531.xml
datasets/road/Annotations/2010_004107.xml
datasets/road/Annotations/2010_005222.xml
['87', '9', '195', '254']
0 0.282 0.3632596685082873 0.216 0.6767955801104972

['284', '209', '346', '287']
0 0.63 0.6850828729281768 0.124 0.2154696132596685

['109', '40', '275', '269']
0 0.384 0.42679558011049723 0.332 0.6325966850828729

datasets/road/TextAnnotations/2010_005222.txt
datasets/road/Annotations/2011_000006.xml
['257', '115', '375', '290']
0 0.632 0.54 0.236 0.4666666666666667

['93', '112', '241', '326']
0 0.334 0.584 0.296 0.5706666666666667

['173', '111', '303', '280']
0 0.476 0.5213333333333333 0.26 0.45066666666666666

['152', '194', '500', '375']
0 0.652 0.75866666

...

['1', '2', '500', '375']
0 0.501 0.5026666666666667 0.998 0.9946666666666667

datasets/road/TextAnnotations/2010_001326.txt
datasets/road/Annotations/2009_004134.xml
datasets/road/Annotations/2008_001182.xml
datasets/road/Annotations/2011_001177.xml
['261', '126', '351', '360']
0 0.612 0.648 0.18 0.624

['278', '126', '311', '162']
0 0.589 0.384 0.066 0.096

['328', '247', '344', '271']
0 0.672 0.6906666666666667 0.032 0.064

['265', '240', '276', '259']
0 0.541 0.6653333333333333 0.022 0.050666666666666665

['298', '337', '323', '359']
0 0.621 0.928 0.05 0.058666666666666666

['263', '340', '291', '360']
0 0.554 0.9333333333333333 0.056 0.05333333333333334

datasets/road/TextAnnotations/2011_001177.txt
datasets/road/Annotations/2011_004312.xml
['406', '171', '446', '283']
0 0.852 0.6837349397590361 0.08 0.3373493975903614

['456', '153', '482', '211']
0 0.938 0.5481927710843374 0.052 0.1746987951807229

['310', '143', '335', '205']
0 0.645 0.5240963855421686 0.05 0.18674698795180722



['200', '32', '456', '333']
0 0.656 0.5480480480480481 0.512 0.9039039039039038

datasets/road/TextAnnotations/2012_003173.txt
datasets/road/Annotations/2010_006455.xml
['82', '53', '325', '375']
0 0.407 0.5706666666666667 0.486 0.8586666666666667

['9', '37', '138', '375']
0 0.147 0.5493333333333333 0.258 0.9013333333333333

datasets/road/TextAnnotations/2010_006455.txt
datasets/road/Annotations/2010_000815.xml
datasets/road/Annotations/2012_001702.xml
['47', '1', '153', '190']
0 0.2 0.25466666666666665 0.212 0.504

['266', '1', '316', '134']
0 0.582 0.18 0.1 0.3546666666666667

['212', '4', '271', '131']
0 0.483 0.18 0.118 0.33866666666666667

datasets/road/TextAnnotations/2012_001702.txt
datasets/road/Annotations/2010_005475.xml
datasets/road/Annotations/2008_002876.xml
datasets/road/Annotations/2010_005388.xml
datasets/road/Annotations/2011_001871.xml
datasets/road/Annotations/2011_005299.xml
['112', '92', '250', '276']
0 0.362 0.49066666666666664 0.276 0.49066666666666664

['339',

['171', '101', '360', '294']
0 0.531 0.5224867724867724 0.378 0.5105820105820106

datasets/road/TextAnnotations/2011_005362.txt
datasets/road/Annotations/2011_002070.xml
['180', '50', '306', '331']
0 0.486 0.508 0.252 0.7493333333333333

['218', '50', '257', '97']
0 0.475 0.196 0.078 0.12533333333333332

['199', '61', '220', '90']
0 0.419 0.20133333333333334 0.042 0.07733333333333334

['252', '309', '272', '328']
0 0.524 0.8493333333333334 0.04 0.050666666666666665

['215', '307', '243', '329']
0 0.458 0.848 0.056 0.058666666666666666

['294', '172', '304', '188']
0 0.598 0.48 0.02 0.042666666666666665

datasets/road/TextAnnotations/2011_002070.txt
datasets/road/Annotations/2009_002415.xml
datasets/road/Annotations/2009_001554.xml
datasets/road/Annotations/2009_001387.xml
datasets/road/Annotations/2011_001726.xml
datasets/road/Annotations/2008_006370.xml
datasets/road/Annotations/2010_005733.xml
['136', '6', '229', '193']
0 0.365 0.2653333333333333 0.186 0.49866666666666665

['1', '114

['334', '222', '346', '237']
0 0.68 0.687125748502994 0.024 0.04491017964071856

['362', '292', '376', '299']
0 0.738 0.8847305389221557 0.028 0.020958083832335328

['335', '290', '349', '298']
0 0.684 0.8802395209580839 0.028 0.023952095808383235

['406', '183', '453', '307']
0 0.859 0.7335329341317365 0.094 0.3712574850299401

['415', '184', '436', '198']
0 0.851 0.5718562874251497 0.042 0.041916167664670656

['442', '196', '450', '205']
0 0.892 0.6002994011976048 0.016 0.02694610778443114

['405', '193', '413', '202']
0 0.818 0.5913173652694611 0.016 0.02694610778443114

['431', '299', '444', '306']
0 0.875 0.905688622754491 0.026 0.020958083832335328

['413', '298', '426', '306']
0 0.839 0.9041916167664671 0.026 0.023952095808383235

datasets/road/TextAnnotations/2009_000067.txt
datasets/road/Annotations/2012_000787.xml
['316', '121', '500', '300']
0 0.816 0.5613333333333334 0.368 0.47733333333333333

['173', '71', '288', '342']
0 0.461 0.5506666666666666 0.23 0.7226666666666667

[

['93', '133', '125', '232']
0 0.218 0.4866666666666667 0.064 0.264

['330', '133', '452', '375']
0 0.782 0.6773333333333333 0.244 0.6453333333333333

['165', '159', '221', '279']
0 0.386 0.584 0.112 0.32

['275', '132', '309', '205']
0 0.584 0.4493333333333333 0.068 0.19466666666666665

datasets/road/TextAnnotations/2012_000596.txt
datasets/road/Annotations/2009_004655.xml
datasets/road/Annotations/2011_005510.xml
['144', '255', '244', '425']
0 0.5173333333333333 0.68 0.26666666666666666 0.34

datasets/road/TextAnnotations/2011_005510.txt
datasets/road/Annotations/2008_003592.xml
datasets/road/Annotations/2011_006162.xml
['5', '12', '351', '307']
0 0.4009009009009009 0.4230769230769231 0.7792792792792793 0.7824933687002652

datasets/road/TextAnnotations/2011_006162.txt
datasets/road/Annotations/2011_006985.xml
['132', '38', '263', '330']
0 0.5238726790450928 0.368 0.34748010610079577 0.584

datasets/road/TextAnnotations/2011_006985.txt
datasets/road/Annotations/2010_000184.xml
datasets

In [143]:
# read all text files with correct objects
annotation_txt_files = glob.glob("datasets/road/TextAnnotations/*.txt")
#print(annotation_txt_files)
for i in range(len(annotation_txt_files)):
    curr_file = annotation_txt_files[i]
    txt_file_name = curr_file.split("/")[-1]
    file_name = txt_file_name.split(".")[0]

    old_image_file_path = ("datasets/road/JPEGImages/" + file_name + ".jpg")
    new_image_file_path = ("datasets/road/Images/" + file_name + ".jpg")
    print(image_file_path)
    shutil.move(old_image_file_path, "datasets/road/Images")

datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
d

datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
d

datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
d

datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
d

datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
d

datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
datasets/road/Images/2010_006092.jpg
d

In [146]:
# creating lists of the train and validation files
file_train = "datasets/road_train.txt"
file_test = "datasets/road_test.txt"

ftrain = open(file_train, "a")
ftest = open(file_test, "a")

text_files = glob.glob("datasets/road/*.txt")

# split list of text files into 80% train and 20% validation
train, validate = np.split(text_files, [int(len(text_files)*0.8)])
#print(len(train), len(validate), len(text_files), len(text_files)*0.2)
for j in range(len(train)):
    train_file_name = train[j].split(".")[0]+".jpg"
#         print(train_file_name)
    ftrain.write(train_file_name+"\n")
for j in range(len(validate)):
    test_file_name = validate[j].split(".")[0]+".jpg"
    ftest.write(test_file_name+"\n")